In [60]:
from scipy.stats import norm
from itertools import combinations
import numpy as np
import math


n_node,n_sample = 6,2000
digma =0.8
X1 = np.random.normal(loc=0.0, scale=digma, size=n_sample)
X2 = 0.5*X1+ np.random.normal(loc=0.0, scale=digma, size=n_sample)
X3 = 0.5*X2+ np.random.normal(loc=0.0, scale=digma, size=n_sample)
X4 = 0.5*X2+ np.random.normal(loc=0.0, scale=digma, size=n_sample)
X5 = 0.5*X3+ 0.5*X4 + np.random.normal(loc=0.0, scale=digma, size=n_sample)
X6 = 0.5*X5+ np.random.normal(loc=0.0, scale=digma, size=n_sample)
data= np.vstack([X1,X2,X3,X4,X5,X6])

var_num = 6
alpha = 0.025
adj_mat = np.ones([6, 6])
cov_ = np.cov(data)


In [61]:
def fisher_Z_test(n_sample, alpha, cond_idx, var_idx, cov_):
        
        comb_var_idx = np.append(var_idx, cond_idx)

        if cond_idx.shape[0] == 0:
            cor_var = cov_[var_idx[0], var_idx[1]]
        else:
            precision_matrix = np.linalg.pinv(cov_[np.ix_(comb_var_idx, comb_var_idx)])
            cor_var = - precision_matrix[0, 1] / np.sqrt(precision_matrix[0, 0] * precision_matrix[1, 1])
        cor_var = min(0.99999, max(cor_var, -0.99999))
        z_trans = 0.5 * math.log((1 + cor_var) / ( 1-cor_var))
        X = math.sqrt(n_sample - comb_var_idx.shape[0] - 3) * abs(z_trans) 
        p_value = 2 * (1 - norm.cdf(abs(X)))
        if p_value > alpha:
            
            return p_value, True
        else:
            return p_value, False
        

def skeleton_discovery(cov_, adj_mat, var_num, n_sample, alpha):
    depth = -1
    while max(np.sum(adj_mat != 0, axis = 1)) - 1 > depth:
        depth += 1
        for x in range(var_num):
            adj_vars, neigh_num = get_neigh(adj_mat, x)
            if neigh_num < depth - 1:
                continue
            for y in adj_vars:
                neigh_x_without_y = np.delete(adj_vars, np.where(adj_vars == y))
                for S in combinations(neigh_x_without_y, depth):
                    _, flag = fisher_Z_test(n_sample, alpha, np.array(S), np.array([x, y]), cov_)
                    if flag:
                        adj_mat[x, y] = 0
                        adj_mat[y, x] = 0
                        
    return adj_mat


def get_neigh(adj_mat, var_idx):
    neigh_set = np.where(adj_mat[var_idx, :] != 0)
    return neigh_set[0], len(neigh_set[0])

def check_cardi(adj_mat, cardinality, var_num):
    for i in range(var_num):
        if np.sum(adj_mat[i, :]) >= cardinality + 1:
            return True # This shows not each ordered pair of adjacent vertices X, Y, Adj(C, X)\{Y} is of cardinality less than n.
    return False


In [64]:
adj = skeleton_discovery(cov_, adj_mat, var_num, n_sample, alpha)

In [65]:
print(np.triu(adj, 1))

[[0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0.]]


In [32]:
from causallearn.search.ConstraintBased.PC import pc

cg = pc(np.transpose(data), alpha)
np.triu(cg.G.graph, 1)

Depth=3, working on node 5: 100%|██████████| 6/6 [00:00<00:00, 1321.04it/s]


array([[ 0, -1,  0,  0,  0,  0],
       [ 0,  0, -1, -1,  0,  0],
       [ 0,  0,  0, -1, -1,  0],
       [ 0,  0,  0,  0, -1,  1],
       [ 0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0]])